# N-body Simulation

## 1. Import libraries

In [1]:
import numpy as np
from numpy import random

import matplotlib.pyplot as plt
from matplotlib import animation
from matplotlib import cm
from mpl_toolkits import mplot3d

plt.rcParams['animation.ffmpeg_path']=r'FFmpeg'
from matplotlib.animation import FFMpegWriter

## 2. Create classes

In [2]:
class Vector:
    def __init__(self, x, y, z):
        self.x = x
        self.y = y
        self.z = z
        
 #add/subtract
    def __add__(self, other):
        return Vector(self.x + other.x, self.y + other.y, self.z + other.z)
    
    def __sub__(self, other):
        return Vector(self.x - other.x, self.y - other.y, self.z - other.z)

 # Scalar Multiplication
    def __mul__(self, scalar):
        return Vector(self.x * scalar, self.y * scalar, self.z * scalar)

    def __rmul__(self, scalar):
        return self.__mul__(scalar)

 #dot product
    def dot(self, other):
        return self.x * other.x + self.y * other.y + self.z * other.z
    

 #cross product (a ** b)
    def __pow__(self, other):
        return Vector(self.y*other.z - self.z*other.y, 
                            self.z*other.x - self.x*other.z, 
                            self.z*other.y - self.y*other.x)
 #magnitude
    def magnitude(self):
        return (self.x**2 + self.y**2 + self.z**2)**(1/2)
    
 #other functions
    def printV(self):
        print((self.x, self.y, self.z))

        
class body:
        def __init__(self, location, velocity, mass):
            self.location = location
            self.velocity = velocity
            self.mass = mass

## 3. Make the bodies

In [13]:
Bulge = body(location = Vector(0,0,0), velocity = Vector(0,0,0), mass = 10e10)

def create_N_bodies(N, N_mass):
    N_bodies = []


    for i in range(N):
        initial_x = random.randint(-40, 40)   #initial x positions are in range (0,40)    #change to 1e4 to 6e4 or whatever
        initial_y = random.randint(-40, 40)   #initial y positions are in range (0,40)
        initial_z = random.randint(-25, 25)   #initial z positions are in range (0,15)

        body_i = body(location = Vector(initial_x, initial_y, initial_z), velocity = Vector(0,0,0), mass = N_mass)
        N_bodies.append(body_i)
        
    return N_bodies

N_bodies = create_N_bodies(100,2e6)   #****CHANGE NUMBER AND MASS OF N BODIES****

## 4. Acceleration, velocity, position functions

In [14]:
def getAcc(N_bodies, body_index):
    softening = 8                   #****CHANGE SOFTENING****
    acceleration = Vector(0,0,0)
    body_i = N_bodies[body_index]
    for index, body_j in enumerate(N_bodies):
        if index != body_index: 
    
            inv_r = ((body_i.location.x - body_j.location.x)**2 + 
                     (body_i.location.y - body_j.location.y)**2 + 
                     (body_i.location.z - body_j.location.z)**2 + 
                     softening**2)**(-3/2)
            a_eqn = 6.67e-11 * body_j.mass * inv_r
        
            acceleration.x += a_eqn * (body_j.location.x - body_i.location.x)
            acceleration.y += a_eqn * (body_j.location.y - body_i.location.y)
            acceleration.z += a_eqn * (body_j.location.z - body_i.location.z)
    
    return acceleration 



def getVel(N_bodies, dt = 1):
    for body_index, body_i in enumerate(N_bodies):
        acceleration = getAcc(N_bodies, body_index)
        
        body_i.velocity.x += acceleration.x * dt
        body_i.velocity.y += acceleration.y * dt
        body_i.velocity.z += acceleration.z * dt
    return None
    
def getPos(N_bodies, dt = 1):
    for body_i in N_bodies:    
        body_i.location.x += body_i.velocity.x * dt
        body_i.location.y += body_i.velocity.y * dt
        body_i.location.z += body_i.velocity.z * dt
    return None
   

def getGravStep(N_bodies, dt = 1):
    getVel(N_bodies, dt = dt)
    getPos(N_bodies, dt = dt)

## 4. Simulation function

In [15]:
def run_sim(N_bodies, dt = 0.1, number_of_steps = 200      #****CHANGE NUMBER OF TIMESTEPS AND dt****
           ):

    locations_hist = []
    
    for body in N_bodies:
        locations_hist.append({"x": [],"y":[],"z":[]})
        
    for i in range(1,number_of_steps):
        getGravStep(N_bodies, dt = 100)        #WHAT TO DO ABOUT THIS DT = 1000 BUSINESS????    
        
        
        for index, location in enumerate(locations_hist):
            location["x"].append(N_bodies[index].location.x)
            location["y"].append(N_bodies[index].location.y)           
            location["z"].append(N_bodies[index].location.z)       

    return locations_hist      

locations_history = run_sim(N_bodies)

## 5. Animation

### 5.1 Make lists

In [16]:
%matplotlib qt

number_of_times = len(locations_history[0]["x"])
number_of_bodies = len(locations_history)

#create empty list
x_list = []
y_list = []
z_list = []

distance = []

#add the data to the empty list
for i in range(number_of_bodies):
    x_list.append(locations_history[i]["x"][0])
    y_list.append(locations_history[i]['y'][0])        
    z_list.append(locations_history[i]['z'][0])
    distance.append(((-40 - locations_history[i]['x'][0])**2 +(-40 - locations_history[i]['y'][0])**2+40)**0.5)

#might be obsolete depending on how i use the animation for loop. hmmm.

### 5.2 2D Animation

In [17]:
metadata = dict(title="copy2danim", artist='asher :)')
writer = FFMpegWriter(fps=15, metadata=metadata)
fig = plt.figure()
ax = fig.add_subplot(111)
ax.patch.set_facecolor('black')


with writer.saving(fig, "copy_2d_again.mp4", dpi=100):
    
    for time_index in range(number_of_times):

        for i in range(number_of_bodies):
            x_list[i] = locations_history[i]['x'][time_index]
            y_list[i] = locations_history[i]['y'][time_index]

        plt.cla()
        plt.xlim([-40, 40])      #change to new range
        plt.ylim([-40, 40])      #change
        ax.patch.set_facecolor('black')
        plt.plot(x_list,y_list,'wo', ms = 1.3)
        ax.set_xlabel('x (kpc)')
        ax.set_ylabel('y (kpc)')
        ax.set_title('2D N-Body Simulation', fontsize=20)
        plt.show()
        plt.draw()
        plt.pause(0.1)
        writer.grab_frame()

### 5.3 3D Animation

In [18]:
colour_map = plt.cm.get_cmap('winter')
inverse_colour_map = colour_map.reversed()



metadata2 = dict(title="copy3danim", artist='asher :)')
writer = FFMpegWriter(fps=15, metadata=metadata2)
fig2 = plt.figure()
ax2 = fig2.add_subplot(111, projection='3d')
ax2.view_init(25, -130)                         #CHANGE VIEW ANGLE
    
with writer.saving(fig2, "final3d.mp4", dpi=100):
            
    for time_index in range(number_of_times):

        for i in range(number_of_bodies):
            x_list[i] = locations_history[i]['x'][time_index]
            y_list[i] = locations_history[i]['y'][time_index]
            z_list[i] = locations_history[i]['z'][time_index]
            distance[i] = ((-40 - locations_history[i]['x'][time_index])**2 +(-40 - locations_history[i]['y'][time_index]-40)**2)**0.5


        fig2.clear()
        ax2 = fig2.gca(projection='3d')  # 3D plot
        ax2.set_xlim(-40, 40); ax2.set_ylim(-40, 40); ax2.set_zlim(-25, 25);                   #CHANGE LIMITS
        ax2.scatter3D(x_list, y_list, z_list, c=distance, cmap=inverse_colour_map);
        ax2.view_init(25, -130)   #CHANGE VIEW ANGLE AGAIN?
        plt.draw()
        plt.pause(0.1)  # runs the frame for 0.01 seconds
        writer.grab_frame()  # this saves the frame
